# OH4VNA Development Notebook

This notebook demonstrates the OH4VNA package functionality and provides examples for development and testing.

In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Add OH4VNA package to path
sys.path.insert(0, str(Path().absolute().parent))

import numpy as np
import matplotlib.pyplot as plt
from oh4vna.services import InstrumentManager
from oh4vna.config import settings

print(f"OH4VNA package loaded successfully!")
print(f"Simulation mode: {settings.simulation_mode}")

## Connect to Instrument

Connect to a VNA instrument (simulation or real hardware).

In [ ]:
# Create instrument manager
manager = InstrumentManager()

# Connect to simulation VNA
success = manager.connect(instrument_type="simulation")
print(f"Connection successful: {success}")

if success:
    info = manager.get_info()
    print(f"Connected to: {info['info']['manufacturer']} {info['info']['model']}")

## Configure and Measure

Set up measurement parameters and perform a sweep.

In [ ]:
# Configure measurement
start_freq = 1e6    # 1 MHz
stop_freq = 6e9     # 6 GHz
points = 201
if_bw = 1000        # Hz
power = -10         # dBm

manager.configure_measurement(
    start_freq=start_freq,
    stop_freq=stop_freq,
    points=points,
    if_bandwidth=if_bw,
    power=power,
    port_count=2
)

print("Measurement configured successfully!")

In [ ]:
# Perform measurement
print("Triggering measurement...")
manager.instrument.trigger_sweep()

print("Waiting for sweep to complete...")
success = manager.instrument.wait_for_sweep(timeout=30.0)

if success:
    print("Getting S-parameter data...")
    network = manager.instrument.get_s_parameters()
    print(f"Measurement complete! Got {len(network.frequency.f)} frequency points")
    print(f"S-parameter matrix shape: {network.s.shape}")
else:
    print("Measurement timeout!")

## Plot Results

Visualize the measured S-parameters.

In [ ]:
# Plot S-parameters
freq_ghz = network.frequency.f / 1e9

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('S-Parameters', fontsize=16)

# S11 magnitude
s11_db = 20 * np.log10(np.abs(network.s[:, 0, 0]))
axes[0, 0].plot(freq_ghz, s11_db, 'b-', label='S11')
axes[0, 0].set_xlabel('Frequency (GHz)')
axes[0, 0].set_ylabel('Magnitude (dB)')
axes[0, 0].set_title('S11 Magnitude')
axes[0, 0].grid(True)

# S21 magnitude
s21_db = 20 * np.log10(np.abs(network.s[:, 1, 0]))
axes[0, 1].plot(freq_ghz, s21_db, 'g-', label='S21')
axes[0, 1].set_xlabel('Frequency (GHz)')
axes[0, 1].set_ylabel('Magnitude (dB)')
axes[0, 1].set_title('S21 Transmission')
axes[0, 1].grid(True)

# S11 phase
s11_phase = np.angle(network.s[:, 0, 0], deg=True)
axes[1, 0].plot(freq_ghz, s11_phase, 'b-', label='S11 Phase')
axes[1, 0].set_xlabel('Frequency (GHz)')
axes[1, 0].set_ylabel('Phase (degrees)')
axes[1, 0].set_title('S11 Phase')
axes[1, 0].grid(True)

# S21 phase
s21_phase = np.angle(network.s[:, 1, 0], deg=True)
axes[1, 1].plot(freq_ghz, s21_phase, 'g-', label='S21 Phase')
axes[1, 1].set_xlabel('Frequency (GHz)')
axes[1, 1].set_ylabel('Phase (degrees)')
axes[1, 1].set_title('S21 Phase')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## Save Data

Export measurement data to Touchstone format.

In [ ]:
# Save to Touchstone file
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"notebook_measurement_{timestamp}.s2p"
filepath = settings.touchstone_dir / filename

# Ensure directory exists
filepath.parent.mkdir(parents=True, exist_ok=True)

# Write Touchstone file
network.write_touchstone(str(filepath))
print(f"Data saved to: {filepath}")

# Also save metadata
import json

metadata = {
    "timestamp": timestamp,
    "measurement_type": "2-port S-parameters",
    "start_freq_hz": start_freq,
    "stop_freq_hz": stop_freq,
    "points": points,
    "if_bandwidth_hz": if_bw,
    "power_dbm": power,
    "instrument_info": manager.get_info()["info"],
    "touchstone_file": filename
}

metadata_file = settings.metadata_dir / f"notebook_measurement_{timestamp}.json"
metadata_file.parent.mkdir(parents=True, exist_ok=True)

with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata saved to: {metadata_file}")

## Cleanup

Disconnect from the instrument when done.

In [ ]:
# Disconnect
manager.disconnect()
print("Disconnected from instrument.")